In [ ]:
# Pancreatic Cancer Detection Notebook

In [2]:
import pandas as pd
import numpy as np

# Data Loading and Preprocessing 

Ideas we should probably do for both datasets:
    - load data. remove any Null/NaN data, see if there are negative values and remove those if applicable.
    - if blank data cells, impute the data. scale the appropriate data columns


    - split the dataset into X and y subsets for passing into models
        - which really means also split into 80/20; 80% training(validation included) and 20% test

    - Note: we should do some data visualizations --> heatmaps? histograms of label distributions for proof
        - of good splits. mutual information??? (need to better understand if its useful)


# Model Creation

- we want to initialize all 4 models, probably default hyperparameters for most values.
    - it's 4 models per dataset, so we'll have 8 in total

# Training and Validation

- Train the models using 10-fold cross validation 
    - we can probably do this in 2 separate cross-validation loops. 
        - first one is for urinary dataset and we train/validate all 4 models (get their scores and loss? too)
        - repeat for second dataset

# Testing Models

- Test each of the models on the appropriate dataset and store values


# Significance Analysis of Results

- Do the McNemar test here for each of the models (8?) to compare

# Comparison Across Datasets

- take urinary dataset models and try to predict on other dataset's test samples and vice versa
    - draw conclusions on the accuracy percentage and based on this, we can determine if the urinary biomarkers
        - are generalizeable to use on other data for predicting pancreatic cancer

# ------------- NON URINARY DATA WORK --------

### Normal pancreas RNA-seq data

In [4]:
# Normal pancreas dataset 1
rawPancNormData1 = pd.read_csv("pancreaticNormalSeqData/GSE205163_znf808_ko_raw_counts_S0-S4.tsv",  sep='\t')
rawPancNormData1

,Gene,WT-H1_S0_1,ZNF808-KO_S0_1,WT-H1_S1_1,ZNF808-KO_S1_1,WT-H1_S2_1,ZNF808-KO_S2_1,WT-H1_S3_1,ZNF808-KO_S3_1,WT-H1_S4_1,...,WT-H1_S0_3,ZNF808-KO_S0_3,WT-H1_S1_3,ZNF808-KO_S1_3,WT-H1_S2_3,ZNF808-KO_S2_3,WT-H1_S3_3,ZNF808-KO_S3_3,WT-H1_S4_3,ZNF808-KO_S4_3
0,ENSG00000000003.15_6|TSPAN6,2121,2514,2457,2630,6593,4822,5811,5019,7145,...,3061,2589,3926,2290,7076,5662,4624,4947,6548,6234
1,ENSG00000000005.6_5|TNMD,25,20,1,3,1,3,0,0,0,...,40,14,1,1,0,0,0,0,0,0
2,ENSG00000000419.13_7|DPM1,1538,1744,992,1107,819,667,719,630,1053,...,1724,1542,1173,801,618,564,645,691,1078,1021
3,ENSG00000000457.14_9|SCYL3,149,180,242,271,359,304,325,358,434,...,185,155,324,223,301,280,253,322,375,354
4,ENSG00000000460.17_8|C1orf112,944,973,447,514,631,561,437,454,389,...,1199,991,471,323,467,380,347,464,361,360
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62475,ENSG00000288669.1_1|AC008763.4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
62476,ENSG00000288670.1_1|AL592295.6,73,71,77,95,108,99,63,77,78,...,60,67,100,89,56,62,57,77,66,85
62477,ENSG00000288671.1_1|AC006486.3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
62478,ENSG00000288674.1_1|AL391628.1,36,32,18,23,26,19,36,37,66,...,11,33,16,4,8,12,24,0,23,13


In [5]:
rawPancNormData2 = pd.read_csv("pancreaticNormalSeqData/GSE216854_normalized_counts.txt", sep='\t')
rawPancNormData2

,gene,GSM6696788,GSM6696789,GSM6696790,GSM6696791,GSM6696792,GSM6696793,GSM6696794,GSM6696795,GSM6696796,...,T21-2wk-3,T21-4wk-1,T21-4wk-2,T21-4wk-3,T21-6wk-1,T21-6wk-2,T21-6wk-3,GSM6696809,GSM6696810,GSM6696811
0,JAG1,90.747385,80.821493,88.182447,141.481386,148.350512,137.692184,356.900130,416.273744,384.389385,...,980.368174,1351.571149,898.731201,852.614996,1599.353004,1100.226616,1087.676421,523.269686,470.409993,466.204249
1,JAG2,2.975324,2.635483,3.391633,5.318849,10.528101,7.510483,9.092997,7.854222,2.308645,...,5.342606,5.636243,10.491026,7.905040,13.315612,11.042420,12.502028,6.782159,4.722992,5.238250
2,SDCBP,1097.894590,1192.995510,1235.684803,1957.336467,1816.575945,1831.306042,1975.453588,2021.480282,2000.440852,...,1904.638986,1712.290721,1989.797873,1570.844317,1140.704131,1604.162529,1331.465964,916.113229,881.310289,924.176963
3,FER1L6-AS1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,FER1L6-AS2,0.743831,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33116,OR2T5-3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
33117,KIR2DL4-30,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
33118,OR4X1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
33119,LINC00698,0.000000,0.000000,0.000000,1.063770,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,1.003856,0.000000,0.000000,0.000000,0.000000


In [6]:
rawPancNormData3 = pd.read_csv("pancreaticNormalSeqData/GSE228662_RNA_raw_read_counts.tsv", sep='\t')
rawPancNormData3

,chrom,start,end,symbol,gene,ABCC8_C1,ABCC8_C2,ABCC8_C3,APOE_C1,APOE_C2,...,WT2_C1,WT2_C2,WT2_C3,WDR13_C1,WDR13_C2,WDR13_C3,WFS1_C1,WFS1_C2,WFS1_C3,WFS1_C4
0,X,100627107,100639991,TSPAN6,ENSG00000000003,633,867,1134,1498,2449,...,347,426,680,1254,794,948,1241,796,1225,1032
1,X,100584935,100599885,TNMD,ENSG00000000005,0,1,0,1,1,...,0,0,0,0,1,0,0,3,0,0
2,20,50934866,50958555,DPM1,ENSG00000000419,685,957,1037,1524,2589,...,464,624,976,960,646,671,1140,872,1063,1071
3,1,169849630,169894267,SCYL3,ENSG00000000457,482,696,675,1101,1739,...,393,455,757,623,408,405,963,847,846,924
4,1,169662006,169854080,C1orf112,ENSG00000000460,134,156,155,271,549,...,44,105,59,179,119,132,217,169,173,190
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67125,4,55346227,55388368,AC064824.1,ENSG00000288695,1,0,1,2,2,...,0,0,0,0,0,1,0,0,0,0
67126,6,163927121,164009980,AL136225.2,ENSG00000288696,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
67127,5,128932069,128932175,AC004636.1,ENSG00000288697,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
67128,3,179604806,179784179,AC007687.1,ENSG00000288698,2,0,0,3,0,...,0,0,0,1,0,1,0,0,0,1


### Pancreatic cancer RNA-seq data

In [43]:
rawPancCancData1 = pd.read_csv("pancreaticCancerSeqData/GSE232860_allsamples.deseq.normalized.counts.csv")
#print(rawPancCancData1.shape)


#Slow loading, 48,553 rows
#rawPancCancData2 = pd.read_excel("pancreaticCancerSeqData/GSE245306_FKPM.xlsx")
#print(rawPancCancData2.shape)

#59050 rows
rawPancCancData3 = pd.read_csv("pancreaticCancerSeqData/tumor.counts.sub.tsv", sep='\t')
#print(rawPancCancData3)

In [65]:
#Only has Reg1, not Reg1B or Reg1A, so we take Reg1 and apply it to both rows
rowsToKeep = ["Gpx1", "Lyve1", "Reg1", "Tff1", "Reg1"]
rawPancCancData1.rename(columns={'Unnamed: 0': 'GeneNames'}, inplace=True)
print(rawPancCancData1.shape)

filtRawPancCancData1 = rawPancCancData1[rawPancCancData1['GeneNames'].isin(rowsToKeep)]
filtRawPancCancData1 = filtRawPancCancData1.transpose()
filtRawPancCancData1.reset_index(inplace=True, drop=True)
filtRawPancCancData1.columns = filtRawPancCancData1.iloc[0]
filtRawPancCancData1 = filtRawPancCancData1[1:]

filtRawPancCancData1["Reg1B"] = filtRawPancCancData1["Reg1"].copy()
filtRawPancCancData1.rename(columns={'Reg1': 'Reg1A'}, inplace=True)

# Impute missing values in each column if needed
filtRawPancCancData1 = filtRawPancCancData1.fillna(filtRawPancCancData1.median())
print(filtRawPancCancData1)





(17966, 23)
0        Tff1          Reg1A       Lyve1          Gpx1          Reg1B
1   86.519650    1152.208802  111.477242   7915.716067    1152.208802
2   99.259037    2032.652444   81.133473   6992.151782    2032.652444
3   95.638935    6535.034757   80.722771   8339.013202    6535.034757
4   42.385798   10158.844770   61.860354   7007.403411   10158.844770
5    6.260937   39103.310850  152.766873  10529.644580   39103.310850
6   33.793363    6810.356633  189.839188   7140.338886    6810.356633
7    4.244787    3628.231888  217.545346   8391.944362    3628.231888
8    2.204342   60100.287620  598.478904   6179.873328   60100.287620
9    4.750487    3313.939555  153.915771   7932.362771    3313.939555
10  21.847610   16311.226750  294.942730   6057.746312   16311.226750
11  42.165889    2393.872495  151.413873   8880.711134    2393.872495
12  35.099226    1474.167487  128.697162   6995.958204    1474.167487
13   0.000000   82462.293950   26.049017   8902.410494   82462.293950
14   7.9